# Vega Tutorial

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from vega import VegaInterface, Wedge
from vega.analysis import Analysis

### Vega can be initializez through the VegaInterface by passing the path to the main.ini config

In [2]:
vega = VegaInterface('main.ini')

INFO: reading input Pk /home/acuceu/work/dev/Vega/vega/models/PlanckDR16/PlanckDR16.fits
LOG: Full matrix is positive definite
LOG: Reduced matrix is positive definite
LOG: Full matrix is positive definite
LOG: Reduced matrix is positive definite


### Run the minimizer and have a look at the output

In [8]:
vega.minimize()

------------------------------------------------------------------
| FCN = 6645                    |      Ncalls=42 (42 total)      |
| EDM = 1.11e-06 (Goal: 0.0002) |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
------------------------------------------------------------------------------------------
|   | Name |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
--------------

In [12]:
vega.bestfit.params

------------------------------------------------------------------------------------------
|   | Name |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------
| 0 | ap   |   1.122   |   0.022   |            |            |   0.1   |    2    |       |
| 1 | at   |   0.931   |   0.027   |            |            |   0.1   |    2    |       |
------------------------------------------------------------------------------------------

## Let's have a look at the structure of the interface

The basic substructure is given by correlation items. There is one of these for each component (corresponding to each config file you input).
These are held in a dictionary with keys given by the names you wrote in the config files. These objects store the information needed to compute a model $\xi$ for that particular component.

In [3]:
vega.corr_items

{'lyaxlya': <vega.correlation_item.CorrelationItem at 0x7f9ebd42f7d0>,
 'qsoxlya': <vega.correlation_item.CorrelationItem at 0x7f9ebc05c5d0>}

They can be initialized on their own by simply passing a parsed config file (such as lyaxlya.ini). Lets see what they contain

In [4]:
vega.corr_items['lyaxlya'].__dict__

{'config': <configparser.ConfigParser at 0x7f9ef0425c90>,
 'name': 'lyaxlya',
 'tracer1': {'name': 'LYA', 'type': 'continuous'},
 'tracer2': {'name': 'LYA', 'type': 'continuous'},
 'has_metals': True,
 'has_bb': False,
 '_rp_rt_grid': array([[  2.07942278,   2.07593782,   2.07702203, ..., 197.97629321,
         197.97381157, 197.97691767],
        [  2.69591934,   6.19556712,  10.13001749, ..., 190.00656533,
         194.0065553 , 198.00786125]]),
 '_r_mu_grid': array([[3.40469972e+00, 6.53410819e+00, 1.03407579e+01, ...,
         2.74403184e+02, 2.77186171e+02, 2.80003523e+02],
        [6.10750713e-01, 3.17707905e-01, 2.00857813e-01, ...,
         7.21479577e-01, 7.14226870e-01, 7.07051523e-01]]),
 '_z_grid': array([2.36224365, 2.36032967, 2.35955733, ..., 2.38663466, 2.38652742,
        2.38631922]),
 'tracer_catalog': {'LYA': {'name': 'LYA', 'type': 'continuous'},
  'CIV(eff)': {'name': 'CIV(eff)', 'type': 'continuous'},
  'SiII(1260)': {'name': 'SiII(1260)', 'type': 'continuous'},


Once we got the the correlation items we can look at the stored data, again stored in dictionaries accessed like the ones above

In [5]:
vega.data

{'lyaxlya': <vega.data.Data at 0x7f9ebe9d6110>,
 'qsoxlya': <vega.data.Data at 0x7f9eb595e210>}

Print what these Data objects contain:

In [6]:
vega.data['lyaxlya'].__dict__

{'_corr_item': <vega.correlation_item.CorrelationItem at 0x7f9ebd42f7d0>,
 '_tracer1': {'name': 'LYA', 'type': 'continuous'},
 '_tracer2': {'name': 'LYA', 'type': 'continuous'},
 'data_vec': array([ 1.00606592e-02,  5.60909335e-03,  3.43969430e-03, ...,
        -7.95319930e-06,  4.42543933e-06, -1.10036449e-05]),
 'cov_mat': array([[ 1.09210301e-08,  3.90300148e-10,  2.12151794e-10, ...,
          4.82806253e-12,  1.69942553e-11,  2.18309495e-11],
        [ 3.90300148e-10,  3.54115358e-09,  1.69256432e-10, ...,
         -7.57252441e-13,  2.68755160e-12,  9.40136911e-12],
        [ 2.12151794e-10,  1.69256432e-10,  2.05379237e-09, ...,
         -1.06198001e-12, -5.63754363e-13,  1.98843357e-12],
        ...,
        [ 4.82806253e-12, -7.57252441e-13, -1.06198001e-12, ...,
          1.06431967e-10,  6.52995172e-12,  4.52793792e-12],
        [ 1.69942553e-11,  2.68755160e-12, -5.63754363e-13, ...,
          6.52995172e-12,  1.01708859e-10,  6.20157078e-12],
        [ 2.18309495e-11,  9.40

All of this data is optional if you just want to play with models, however we still need to define some coordinate grids before we go to model computation.

In [7]:
# For example:
rp_vals = np.linspace(0,196,50) + 2
rt_vals = np.linspace(0,196,50) + 2

rt_grid, rp_grid = np.meshgrid(rp_vals, rt_vals)

# Overwrite rp_rt grid:
# vega.corr_items['lyaxlya'].rp_rt_grid = [rp_grid.flatten(), rt_grid.flatten()]

# Additionally you can specify the r_mu grid:
# vega.corr_items['lyaxlya'].r_mu_grid = [r_grid.flatten(), mu_grid.flatten()]

# Overwrite z grid. This can also take a float
# vega.corr_items['lyaxlya'].z_grid = lyafit.fiducial['z_eff']

Only one of rp/rt and r/mu grids should be specified as the other is automatically computed under the hood

## Next, lets look at how we compute a model

The interface stores Model objects in a dictionary for each component, equivalent to what we've seen above

In [8]:
vega.models

{'lyaxlya': <vega.model.Model at 0x7f9ebc07a710>,
 'qsoxlya': <vega.model.Model at 0x7f9ebc07a550>}

These Model objects store PowerSpectra and CorrelationFunction objects for the core components and for all the metal correlations

In [9]:
vega.models['lyaxlya'].__dict__

{'_corr_item': <vega.correlation_item.CorrelationItem at 0x7f9ebd42f7d0>,
 '_coords_grid': {'r': array([  3.40469972,   6.53410819,  10.34075795, ..., 274.40318427,
         277.18617131, 280.00352328]),
  'mu': array([0.61075071, 0.3177079 , 0.20085781, ..., 0.72147958, 0.71422687,
         0.70705152]),
  'z': array([2.36224365, 2.36032967, 2.35955733, ..., 2.38663466, 2.38652742,
         2.38631922])},
 '_data': <vega.data.Data at 0x7f9ebe9d6110>,
 '_full_shape': False,
 '_smooth_scaling': False,
 '_has_distortion_mat': True,
 'save_components': False,
 'bb_config': None,
 'Pk_core': <vega.power_spectrum.PowerSpectrum at 0x7f9ebc07a650>,
 'Xi_core': <vega.correlation_func.CorrelationFunction at 0x7f9ebd8a4b90>,
 'Pk_metal': {('LYA',
   'CIV(eff)'): <vega.power_spectrum.PowerSpectrum at 0x7f9ebc07a9d0>,
  ('LYA', 'SiII(1260)'): <vega.power_spectrum.PowerSpectrum at 0x7f9ebc07ac90>,
  ('LYA',
   'SiIII(1207)'): <vega.power_spectrum.PowerSpectrum at 0x7f9ebc07a350>,
  ('LYA', 'SiII(11

The PowerSpectrum and CorrelationFunction objects each have many functions of the form "compute_x()". These can be called from outside and will not modify the internal state of the object. E.g.:

In [11]:
print(vega.models['lyaxlya'].Pk_core.compute_dnl_arinyo(vega.params))

[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]


These are all the other options currently available for the PowerSpectrum:

In [17]:
vega.models['lyaxlya'].Pk_core.compute_kaiser
vega.models['lyaxlya'].Pk_core.compute_peak_nl
vega.models['lyaxlya'].Pk_core.compute_bias_beta_hcd
vega.models['lyaxlya'].Pk_core.compute_bias_beta_uv
vega.models['lyaxlya'].Pk_core.compute_dnl_mcdonald
vega.models['lyaxlya'].Pk_core.compute_dnl_arinyo
vega.models['lyaxlya'].Pk_core.compute_fullshape_exp_smoothing
vega.models['lyaxlya'].Pk_core.compute_fullshape_gauss_smoothing
vega.models['lyaxlya'].Pk_core.compute_Gk
vega.models['lyaxlya'].Pk_core.compute_velocity_dispersion_gauss
vega.models['lyaxlya'].Pk_core.compute_velocity_dispersion_lorentz

<bound method PowerSpectrum.compute_bias_beta_hcd of <vega.power_spectrum.PowerSpectrum object at 0x7f9ebc07a650>>

And for the CorrelationFunction:

In [18]:
vega.models['lyaxlya'].Xi_core.compute_bias_evol
vega.models['lyaxlya'].Xi_core.compute_broadband
vega.models['lyaxlya'].Xi_core.compute_core
vega.models['lyaxlya'].Xi_core.compute_growth
vega.models['lyaxlya'].Xi_core.compute_qso_radiation
vega.models['lyaxlya'].Xi_core.compute_xi_asymmetry
vega.models['lyaxlya'].Xi_core.compute_xi_relativistic

<bound method CorrelationFunction.compute_xi_relativistic of <vega.correlation_func.CorrelationFunction object at 0x7f9ebd8a4b90>>

If you want to manually compute a full model correlation function for the core or for one of the metal correlations, you can do something like this:

In [ ]:
# Compute core model correlation function
# k, muk, pk_model = self.Pk_core.compute(pk_lin, pars)
# xi_model = self.Xi_core.compute(k, muk, pk_model, pk_lin, pars)

If you don't want to have to specify the fiducial Pks, you can find them in the fiducial dictionary:

In [20]:
vega.fiducial.keys()

dict_keys(['z_fiducial', 'Omega_m', 'Omega_de', 'k', 'pk_full', 'pk_smooth', 'full-shape', 'smooth-scaling', 'z_eff'])

### If you don't want to do all this work to manually compute individual components, Vega can also automatically save everything for you, when computing the full model:

In [22]:
vega.fiducial['save-components'] = True
bestfit_models = vega.compute_model(vega.params)

This will store 3 big dictionaries in the Model objects: pk, xi and xi_distorted. These all have 'peak' and 'smooth' components

In [27]:
vega.models['lyaxlya'].pk['peak']
vega.models['lyaxlya'].xi['peak']
vega.models['lyaxlya'].xi_distorted['peak']
vega.models['lyaxlya'].pk['smooth']
vega.models['lyaxlya'].xi['smooth']
vega.models['lyaxlya'].xi_distorted['smooth']

{('LYA', 'CIV(eff)'): array([0., 0., 0., ..., 0., 0., 0.]),
 ('LYA',
  'SiII(1260)'): array([-2.88540605e-06, -3.00625103e-06, -2.89202783e-06, ...,
        -1.82416766e-08, -1.57182585e-08, -1.38744672e-08]),
 ('LYA',
  'SiIII(1207)'): array([-1.62354413e-05, -1.13206508e-05, -3.69433250e-06, ...,
        -5.46471667e-08, -5.11270796e-08, -4.16358196e-08]),
 ('LYA',
  'SiII(1193)'): array([-1.02848606e-05, -1.04009385e-05, -9.62094729e-06, ...,
        -5.61748097e-08, -5.14135919e-08, -3.20297996e-08]),
 ('LYA',
  'SiII(1190)'): array([-8.57700523e-06, -8.73718169e-06, -8.18195245e-06, ...,
        -5.43705564e-08, -4.96404092e-08, -2.89908666e-08]),
 ('CIV(eff)', 'LYA'): array([0., 0., 0., ..., 0., 0., 0.]),
 ('SiII(1260)',
  'LYA'): array([-2.88540605e-06, -3.00625103e-06, -2.89202783e-06, ...,
        -1.82416766e-08, -1.57182585e-08, -1.38744672e-08]),
 ('SiIII(1207)',
  'LYA'): array([-1.62354413e-05, -1.13206508e-05, -3.69433250e-06, ...,
        -5.46471667e-08, -5.11270796e-0

Each of the dictionaries above stores all the individual components from each correlation indexed using 'core' or ('tracer1', 'tracer2') for metal correlations:

In [28]:
vega.models['lyaxlya'].pk['smooth'].keys()

dict_keys(['core', ('LYA', 'CIV(eff)'), ('LYA', 'SiII(1260)'), ('LYA', 'SiIII(1207)'), ('LYA', 'SiII(1193)'), ('LYA', 'SiII(1190)'), ('CIV(eff)', 'LYA'), ('SiII(1260)', 'LYA'), ('SiIII(1207)', 'LYA'), ('SiII(1193)', 'LYA'), ('SiII(1190)', 'LYA'), ('CIV(eff)', 'CIV(eff)'), ('CIV(eff)', 'SiII(1260)'), ('CIV(eff)', 'SiIII(1207)'), ('CIV(eff)', 'SiII(1193)'), ('CIV(eff)', 'SiII(1190)'), ('SiII(1260)', 'SiII(1260)'), ('SiII(1260)', 'SiIII(1207)'), ('SiII(1260)', 'SiII(1193)'), ('SiII(1260)', 'SiII(1190)'), ('SiIII(1207)', 'SiIII(1207)'), ('SiIII(1207)', 'SiII(1193)'), ('SiIII(1207)', 'SiII(1190)'), ('SiII(1193)', 'SiII(1193)'), ('SiII(1193)', 'SiII(1190)'), ('SiII(1190)', 'SiII(1190)')])